In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

In [ ]:
!pip install --pre deepchem
import deepchem
deepchem.__version__

In [ ]:
!pip install -q mdtraj nglview

In [1]:
import os
import numpy as np
import pandas as pd

import tempfile

from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc

from deepchem.utils import download_url, load_from_disk

In [6]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!mamba install openmm

In [ ]:
!conda install pdbfixer

In [2]:
from openmm.app import PDBFile
from pdbfixer import PDBFixer

from deepchem.utils.vina_utils import prepare_inputs

In [ ]:
data_dir = dc.utils.get_data_dir()
dataset_file = os.path.join(data_dir, "pdbbind_core_df.csv.gz")

if not os.path.exists(dataset_file):
    print('File does not exist. Downloading file...')
    download_url("https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/pdbbind_core_df.csv.gz")
    print('File downloaded...')

raw_dataset = load_from_disk(dataset_file)
raw_dataset = raw_dataset[['pdb_id', 'smiles', 'label']]

File does not exist. Downloading file...
File downloaded...


In [ ]:
print(raw_dataset['smiles'].iloc[1])

CC(C)(C)NC(O)C1CC2CCCCC2C[NH+]1CC(O)C(CC1CCCCC1)NC(O)C(CC(N)O)NC(O)C1CCC2CCCCC2N1


In [3]:
pdbid  = '1XQ8'
ligand = 'S(=O)(=O)(N1CCNCCC1)C1C2C(CCC1)CCCC2'

pdbid  = '1M47'
ligand = 'CC(CC(C(=O)NCC(=O)N1CCC(CC1)c1cc(nn1C)c1ccc(c(c1Cl)Cl)OCc1ccc(o1)C(=O)O)N=C(N)N)C'
ligand ='NN=CN1CCCC(C1)CC(=O)NC(C(=O)OC)Cc1ccc(cc1)C#Cc1ccccc1'

In [4]:
featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
f = featurizer.featurize(ligand, sanitize = True)

In [5]:
%%time
fixer = PDBFixer(pdbid=pdbid)
PDBFile.writeFile(fixer.topology, fixer.positions, open('%s.pdb' % (pdbid), 'w'))

p, m = None, None
#p= None
# fix protein, optimize ligand geometry, and sanitize molecules
try:
    p, m = prepare_inputs('%s.pdb' % (pdbid), ligand)
    #p= prepare_inputs('%s.pdb' % (pdbid))
except:
    print('%s failed PDB fixing' % (pdbid)) 

if p and m:  # protein and molecule are readable by RDKit
    print(pdbid, p.GetNumAtoms())
    Chem.rdmolfiles.MolToPDBFile(p, '%s.pdb' % (pdbid))
    Chem.rdmolfiles.MolToPDBFile(m, 'ligand_%s.pdb' % (pdbid))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  


1M47 999
CPU times: user 1.03 s, sys: 236 ms, total: 1.26 s
Wall time: 1.74 s


In [6]:
!ls

1M47.pdb		ligand_1M47.pdb			   sample_data
1XQ8.pdb		ligand_1XQ8.pdb			   vina_test
condacolab_install.log	Miniconda3-latest-Linux-x86_64.sh
conda_installer.py	__pycache__


In [7]:
import mdtraj as md
import nglview

from IPython.display import display, Image

protein_mdtraj = md.load_pdb('1XQ8.pdb')
ligand_mdtraj = md.load_pdb('ligand_1XQ8.pdb')


protein_mdtraj = md.load_pdb('1M47.pdb')
ligand_mdtraj = md.load_pdb('ligand_1M47.pdb')

In [8]:
from google.colab import output
output.enable_custom_widget_manager()
v = nglview.show_mdtraj(protein_mdtraj)
display(v)

NGLWidget()

In [9]:
ll = nglview.show_mdtraj(ligand_mdtraj)
display(ll)

NGLWidget()

In [10]:
finder = dc.dock.binding_pocket.ConvexHullPocketFinder()
pockets = finder.find_pockets('1M47.pdb')
len(pockets)  # number of identified pockets

43

In [11]:
vpg = dc.dock.pose_generation.VinaPoseGenerator()

In [12]:
!mkdir -p vina_test

In [ ]:
!conda install vina

In [13]:
%%time
complexes, scores = vpg.generate_poses(molecular_complex=('1M47.pdb', 'ligand_1M47.pdb'),  # protein-ligand files for docking,
                                       out_dir='vina_test',
                                       generate_scores=True
                                      )

Mol [H]N[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)N([H])[C@]([H])(C(=O)O)[C@]([H])(O[H])C([H])([H])[H])C([H])([H])C([H])(C([H])([H])[H])C([H])([H])[H])[C@]([H])(O[H])C([H])([H])[H])C([H])([H])O[H])[C@@]([H])(C([H])([H])[H])C([H])([H])C([H])([H])[H])[C@@]([H])(C([H])([H])[H])C([H])([H])C([H])([H])[H])C([H])([H])O[H])C([H])([H])C([H])([H])C(=O)N([H])[H])C([H])([H])S[H])C([H])([H])C1=C([H])C([H])=C([H])C([H])=C1[H])[C

CPU times: user 7min 56s, sys: 2.91 s, total: 7min 59s
Wall time: 4min 7s


In [14]:
scores


[-5.791, -5.744, -5.651, -5.571, -5.486, -5.443, -5.36, -5.317, -5.283]

In [15]:
from google.colab import output
output.enable_custom_widget_manager()
complex_mol = Chem.CombineMols(complexes[0][0], complexes[0][1])
v = nglview.show_rdkit(complex_mol)
display(v)

NGLWidget()

!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

!pip install --pre deepchem
import deepchem
deepchem.__version__

!pip install -q mdtraj nglview

!pip install -q condacolab
import condacolab
condacolab.install()

!mamba install openmm
!conda install pdbfixer